# TIES581 Project work

Mikael Myyrä  
`mikael.b.myyra@jyu.fi`

In this document I implement and test Krylov subspace methods for
linear systems using NumPy, based on the descriptions of Saad (2003).

# Test problems and library setup

Using numpy for matrix utilities and scipy to read the Harwell-Boeing matrix format.

The matrices are picked from the Harwell-Boeing and FIDAP collections on
[Matrix Market](https://math.nist.gov/MatrixMarket/).
Two of them, ORSIRR1 and FIDAP36, are also used by Saad (2003).
FIDAP005 is a smaller problem that is helpful in early testing
because you can print and read it.

Following Saad (2003) chapter 3.7, the right-hand side of $Ax = b$ is generated
as $b = Ae$ where $e = (1,1,\dots,1)^T$, and the initial guess $x_0$
is a vector of random values. Saad does not specify the range
or distribution of random values, so I am assuming the conventional
uniform distribution in the range $[0, 1)$.

In [1]:
import numpy as np
import scipy as sp
from scipy.io import mmread

import math
import time
from dataclasses import dataclass
from typing import Callable
from typing import Union

def default_rhs(A) -> np.ndarray:
    return A * np.ones((A.shape[1],))

def random_guess(A) -> np.ndarray:
    return np.random.random_sample((A.shape[1],))

@dataclass
class Equation:
    name: str
    A: np.ndarray
    b: np.ndarray

TEST_EQUATIONS: list[Equation] = [
    Equation(name=name, A=mat, b=default_rhs(mat))
    for name, mat in [
        ("FIDAP005", mmread("test_matrices/fidap005.mtx")),
        ("FIDAP036", mmread("test_matrices/fidap036.mtx")),
        ("GR3030", mmread("test_matrices/gr_30_30.mtx")),
        ("ORSIRR1", mmread("test_matrices/orsirr_1.mtx")),
    ]
]

MAX_ITERATIONS = 300

@dataclass
class RunResult:
    ans: np.ndarray
    iterations: int

def test_run(method: Callable[[Equation, np.ndarray], np.ndarray], **kwargs):
    """Run the given method on all the test problems and print statistics."""

    # pretty-printing results as a table
    CELL_SIZE = 16
    def fmt_cell(x: Union[str, float]) -> str:
        text = f"{x:.3e}" if isinstance(x, float) else str(x)
        return text.center(CELL_SIZE)
    def print_row(cells: list[Union[str, float]]):
        print("|".join([fmt_cell(c) for c in cells]))

    headers = ["matrix", "residual norm", "iterations", "time (ms)"]
    print_row(headers)
    print_row(["-" * CELL_SIZE] * len(headers))
    for eq in TEST_EQUATIONS:
        start_time = time.perf_counter_ns()
        result = method(eq, random_guess(eq.A), **kwargs)
        duration_ns = time.perf_counter_ns() - start_time
        duration_ms = duration_ns // 1000000
        resid = np.linalg.norm(eq.b - eq.A * result.ans)
        print_row([eq.name, resid, result.iterations, duration_ms])

np.set_printoptions(precision=5)

# Methods

## Full Orthogonalization Method (FOM)

Saad (2003) presents two variants of FOM, the Restarted and Incomplete versions.
Restarted is a little simpler, so I will try it first.

### Restarted FOM

In [2]:
def restarted_fom(eq: Equation, x0: np.ndarray, subsp_dim: int) -> RunResult:
    """Approximately solve `Ax = b` using the Full Orthogonalization Method
    with Krylov subspace dimension `subsp_dim`."""

    # Krylov subspace can't be larger than the column count of A
    if eq.A.shape[1] < subsp_dim:
        subsp_dim = eq.A.shape[1]

    # stop condition from Saad (2003): reduce residual norm by a factor of 10^7
    initial_resid_norm = np.linalg.norm(eq.b - eq.A * x0)
    stop_resid_limit = 1e-7 * initial_resid_norm
    x = x0
    iter_count = 0
    while iter_count < MAX_ITERATIONS:
        iter_count += 1
        # current residual
        resid = eq.b - eq.A * x
        resid_norm = np.linalg.norm(resid)
        # stop if we're close enough to the correct answer
        # or the algorithm failed and diverged to infinity (this happens with FIDAP036)
        if resid_norm < stop_resid_limit or math.isinf(resid_norm):
            break
        # orthonormal basis of the Krylov subspace,
        # filled in over the course of the algorithm
        V = np.zeros((eq.A.shape[1], subsp_dim))
        # the Hessenberg matrix H in the relation (V^T)AV = H
        H = np.zeros((subsp_dim, subsp_dim))
        # first basis vector of the Krylov subspace based on the residual
        V[:,0] = resid / resid_norm

        for col in range(subsp_dim):
            # the next vector in the Krylov subspace's basis
            w = eq.A * V[:,col]
            # orthogonalize using Modified Gram-Schmidt
            for prev_col in range(col+1):
                H[prev_col, col] = w.dot(V[:,prev_col])
                w -= H[prev_col, col] * V[:,prev_col]
            if col+1 == subsp_dim:
                break
            H[col+1, col] = np.linalg.norm(w)
            if H[col+1, col] < 1e-10:
                # terminated early because the Krylov subspace's actual dimension
                # is less than what was given as parameter.
                # resize the matrices H and V so that they're not singular if this happens
                H = H[:col+1, :col+1]
                V = V[:, :col+1]
                break
            V[:,col+1] = w / H[col+1, col]

        # Using a prebuilt routine for this part for now.
        # I know Saad (2003) has a method for this, but that seems less relevant than
        # trying many different methods, so I'll leave it for later if I have time.
        h_rhs = np.zeros((H.shape[1], 1))
        h_rhs[0] = resid_norm
        y = np.linalg.solve(H, h_rhs)

        # y and its product with V are 2D column vectors,
        # but the rest of the code works in 1D vectors
        x += np.dot(V, y)[:,0]

    return RunResult(
        ans=x,
        iterations=iter_count,
    )

In [3]:
for dim in [10, 30, 50]:
    print(f"Subspace dimension: {dim}")
    test_run(restarted_fom, subsp_dim=dim)
    print("")

Subspace dimension: 10
     matrix     | residual norm  |   iterations   |   time (ms)    
----------------|----------------|----------------|----------------
    FIDAP005    |   1.806e+01    |      300       |      169       
    FIDAP036    |   1.031e+03    |      300       |      803       
     GR3030     |   4.113e-06    |       14       |       11       
    ORSIRR1     |   2.071e+01    |      300       |      360       

Subspace dimension: 30
     matrix     | residual norm  |   iterations   |   time (ms)    
----------------|----------------|----------------|----------------
    FIDAP005    |   1.271e-09    |       2        |       3        
    FIDAP036    |   1.838e+39    |      300       |      2716      
     GR3030     |   3.115e-08    |       4        |       15       
    ORSIRR1     |   3.712e-02    |       64       |      318       

Subspace dimension: 50
     matrix     | residual norm  |   iterations   |   time (ms)    
----------------|----------------|-----------

# Sources

Saad, Y. (2003). Iterative Methods for Sparse Linear Systems.